In [19]:
#This library is used to download contents of a webpage or get data from API
import requests

#Used along with requests to scrape and extract data from web pages
from bs4 import BeautifulSoup

#To interact with SQLite databases
import sqlite3

#To handle and analyze structured data
import pandas as pd

#To work with date and time objects
from datetime import datetime

#To access time-related functions
import time

In [20]:
#Create a database and table if it doesn’t exist
def init_db():
    conn = sqlite3.connect('news_articles.db')
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS articles(
                        article_id TEXT PRIMARY KEY,
                        url TEXT,
                        title TEXT,
                        publication_ts TEXT,
                        source TEXT)''')
    conn.commit()
    conn.close()

In [21]:
#To scrape articles from the homepage of [Skift](https://skift.com/) and return them as a list of dictionaries
def scrape_skift():
    url = "https://skift.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []
    
    for tag in soup.select('article'):
        a_tag = tag.find('a', href=True)
        title_tag = tag.find(['h2', 'h3'])
        date_tag = tag.find('time')
        
        if a_tag and title_tag and date_tag:
            article_url = a_tag['href']
            title = title_tag.text.strip()
            date = date_tag.get('datetime', datetime.now().isoformat())
            article_id = article_url  # Use URL as unique ID
            
            articles.append({
                'article_id': article_id,
                'url': article_url,
                'title': title,
                'publication_ts': date,
                'source': 'Skift'
            })
    return articles

In [22]:
#To extract recent articles from [PhocusWire](https://www.phocuswire.com/) and return them in structured format.
def scrape_phocuswire():
    url = "https://www.phocuswire.com/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = []

    for tag in soup.select('.article-list .article-item'):
        a_tag = tag.find('a', href=True)
        title_tag = tag.find('h2')
        date_tag = tag.find('time')

        if a_tag and title_tag and date_tag:
            article_url = "https://www.phocuswire.com" + a_tag['href']
            title = title_tag.text.strip()
            date = date_tag.get('datetime', datetime.now().isoformat())
            article_id = article_url

            articles.append({
                'article_id': article_id,
                'url': article_url,
                'title': title,
                'publication_ts': date,
                'source': 'PhocusWire'
            })
    return articles

In [23]:
#Store new articles
def store_articles(articles):
    conn = sqlite3.connect('news_articles.db')
    cursor = conn.cursor()

    for article in articles:
        try:
            cursor.execute('''
                INSERT INTO articles (article_id, url, title, publication_ts, source)
                VALUES (?, ?, ?, ?, ?)
            ''', (
                article['article_id'],
                article['url'],
                article['title'],
                article['publication_ts'],
                article['source']
            ))
        except sqlite3.IntegrityError:
            # Duplicate article_id; skip
            continue

    conn.commit()
    conn.close()

In [24]:
#Fetch Top 5 Latest Articles
def get_top_articles(n=5):
    conn = sqlite3.connect('news_articles.db')
    df = pd.read_sql_query('''
        SELECT * FROM articles ORDER BY publication_ts DESC LIMIT ?
    ''', conn, params=(n,))
    conn.close()
    return df

In [25]:
#Pipeline execution
def run_pipeline():
    print("Initializing DB...")
    init_db()
    
    print("Scraping Skift...")
    skift_articles = scrape_skift()
    
    print("Scraping PhocusWire...")
    phocus_articles = scrape_phocuswire()

    print("Storing articles...")
    all_articles = skift_articles + phocus_articles
    store_articles(all_articles)

    print("Top 5 Latest Articles:")
    latest = get_top_articles()
    print(latest[['title', 'publication_ts', 'source']])

In [26]:
if __name__ == "__main__":
    run_pipeline()

Initializing DB...
Scraping Skift...
Scraping PhocusWire...
Storing articles...
Top 5 Latest Articles:
                                               title  \
0  Air India Crash: A Day Later, What We Know So Far   
1  Air India Crash Deepens Traveler Anxiety Aroun...   
2  Tripadvisor Is Turning 25. Here’s What Its CEO...   
3  The Private Sector’s Role in Saudi’s Travel Am...   
4  India’s Expanding Airline Fleets: How Boeing a...   

              publication_ts source  
0  2025-06-13T10:03:47-04:00  Skift  
1  2025-06-13T09:57:54-04:00  Skift  
2  2025-06-13T09:51:44-04:00  Skift  
3  2025-06-13T09:27:05-04:00  Skift  
4  2025-06-13T09:10:53-04:00  Skift  
